<a href="https://colab.research.google.com/github/sungjin-kim-data/ML/blob/master/ridge%2C_lasso_regerssion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import warnings
warnings.filterwarnings(action='ignore')

### Ridge Regression

- Ridge Regression은 회귀계수에 `가중치들의 제곱합(L2 penalty)`을 패널티로 부과하여 회귀계수의 크기를 줄이는 모델입니다.
- L2 패널티를 적용하면 영향력이 크지 않은 회귀계수의 값은 0에 가까운 수로 축소합니다.
- Ridge Regression의 비용함수는 다음과 같습니다.

$$ cost = RSS + \lambda\sum_{j=1}^p\beta_j^2 = \sum_{i=1}^n(y_i - \beta_0 - \beta_1x_{i1}-\dotsc-\beta_px_{ip})^2 + \lambda\sum_{j=1}^p\beta_j^2 $$


### Lasso Regression

- Lasso Regression은 회귀계수에 `가중치들의 절대값의 합(L1 penalty)`을 패널티로 부과하여 회귀계수의 크기를 줄이는 모델입니다.
- Lasso의 중요한 특징은 영향력이 크지 않은 회귀계수의 값을 0으로 만든다는 것입니다.
    - 그 결과 자동으로 특성을 선택하는 효과를 가지게 되며 희소모델(sparse model)을 만들게 됩니다. 
    - 희소 모델이란 가중치가 0인 특성이 많은 모델을 말합니다.
- Lasso Regression의 비용함수는 다음과 같습니다.

$$ cost = RSS + \lambda\sum_{j=1}^p\vert\beta_j\vert = \sum_{i=1}^n(y_i - \beta_0 - \beta_1x_{i1}-\dotsc-\beta_px_{ip})^2 + \lambda\sum_{j=1}^p\vert\beta_j\vert $$

### $\lambda$ (Lambda)
- $\lambda$ 는 패널티의 강도를 조절하는 하이퍼파라미터입니다. lambda, alpha, penalty term, regularization term 등으로 불립니다.
- $\lambda$ 의 크기가 클수록 회귀계수의 값이 줄어듭니다. 
    - $\lambda = 0$ 인 경우 기존의 선형회귀와 같아집니다.
    - $\lambda = \infty $ 인 경우  $\beta = 0$ 이 됩니다.


In [ ]:
ans = sns.load_dataset('anscombe').query('dataset=="III"')
ans.plot.scatter('x', 'y');

In [ ]:
# ridge regression 구현
ridge = Ridge(alpha=0.5, normalize=True) # alpha : 패널티, normalize : 표준화 작업
ridge.fit(ans[['x']], ans['y'])
y_pred_ridge = ridge.predict(ans[['x']])

# 그래프 그리기
ans.plot.scatter('x', 'y')
plt.plot(ans['x'], y_pred_ridge)
plt.title(f'y= {ridge.coef_[0].round(2)}x + {ridge.intercept_.round(2)}')

plt.show()


In [ ]:
# lasso regression 구현
lasso = Lasso(alpha=0.1, normalize=True) # alpha : 패널티, normalize : 표준화 작업
lasso.fit(ans[['x']], ans['y'])
y_pred_lasso = lasso.predict(ans[['x']])


# 그래프 그리기
ans.plot.scatter('x', 'y')
plt.plot(ans['x'], y_pred_lasso)
plt.title(f'y= {lasso.coef_[0].round(2)}x + {lasso.intercept_.round(2)}')
plt.show()ㅠ

In [ ]:
# 그래프를 그리기 위한 함수
def predict_ypred(alpha, model) : 
    ml = model(alpha=alpha, normalize=True)
    ml.fit(ans[['x']], ans['y'])
    m = ml.coef_[0].round(2)
    b = ml.intercept_.round(2)
    formula = f'y = {m}x + {b}'
    ans['y_pred'] = ml.predict(ans[['x']])
    return ans['y_pred'], formula


def plotting(model, alphas, ax, title, legend) :
    colors = ['blue', 'red','orange', 'green']
    for alpha, color in zip(alphas, colors):
        ans['y_pred'], formula = predict_ypred(alpha, model)
        ans.plot('x', 'y_pred', ax=ax, color =color, linestyle='dashed', alpha=0.5)
        ax.text(14, ans.loc[27, 'y_pred'], formula)
    ax.set_title(title)
    ax.legend(legend)

In [ ]:
%matplotlib inline
ans = sns.load_dataset('anscombe').query('dataset=="III"')
ans = ans.sort_values(by='x')

fig = plt.figure(figsize=(30, 6))
ax = fig.add_subplot(131)
ax.scatter(ans['x'], ans['y'])

"""
OLS
"""
ols = LinearRegression()
ols.fit(ans[['x']], ans['y'])

# 회귀계수와 intercept를 확인합니다.
m = ols.coef_[0].round(2)
b = ols.intercept_.round(2)
title = f'Linear Regression \n y = {m}x + {b}'

# 훈련 데이터로 예측을 합니다.
ans['y_pred'] = ols.predict(ans[['x']])
ans.plot('x', 'y_pred', ax=ax, title=title);

"""
Ridge Regression
"""
ax1 = fig.add_subplot(132)
ax1.scatter(ans['x'], ans['y'])

alphas = np.arange(0, 2, 0.5)
title = 'Ridge Regression'
legend = ['ancombe','alpha : 0.0', 'alpha:0.5', 'alpha:1.0', 'alpha:1.5']
plotting(Ridge, alphas, ax1, title, legend)

"""
Lasso Regression
"""
ax2 = fig.add_subplot(133)
ax2.scatter(ans['x'], ans['y'])

alphas = np.arange(0, 0.4, 0.1)
title = 'Lasso Regression'
legend = ['ancombe','alpha : 0.0', 'alpha:0.1', 'alpha:0.2', 'alpha:0.3']
plotting(Lasso, alphas, ax2, title, legend)

plt.show()

그래프를 보면,  
alpha = 0인 경우에는 Ridge와 Lasso 모두 OLS 와 같은 그래프 형태로 같은 모델임을 확인 할 수 있고.
alpha 값이 커질 수록 직선의 기울기가 0에 가까워지는 것을 알 수 있습니다.
정규화모델은 OLS 모델에 비해 이상치에 영향을 덜 받는 것을 알 수 있습니다.